In [1]:
import geopandas as gpd
import pandas as pd
import fiona

In [2]:
soils = gpd.read_file('D:\PythonProgramming\lab1.gpkg')
layers = fiona.listlayers('D:\PythonProgramming\lab1.gpkg')
print(layers)

['soilmu_a_co001', 'soilmu_a_co618', 'soilmu_a_co641', 'soilmu_a_co642', 'soilmu_a_co643', 'soilmu_a_co644', 'soilmu_a_co645', 'soilmu_a_co651', 'soilmu_a_co653', 'wbdhu8_lab1', 'muaggatt_co001', 'muaggatt_co618', 'muaggatt_co641', 'muaggatt_co642', 'muaggatt_co643', 'muaggatt_co644', 'muaggatt_co645', 'muaggatt_co651', 'muaggatt_co653']


In [3]:
polygons = [] 
tables = []

for layer_name in layers:
    if layer_name.startswith('soilmu'):
        polygons.append(layer_name)
for layer_name in layers:
    if layer_name.startswith('muaggatt'):
        tables.append(layer_name)
#seperated the shapefiles from the attribute tables

In [4]:
for spatial in polygons:
    layer_id = spatial [-5:]
    for table in tables:
        if table[-5:] == layer_id:
            print(spatial, table)
# matched the layers based on their layer ID

soilmu_a_co001 muaggatt_co001
soilmu_a_co618 muaggatt_co618
soilmu_a_co641 muaggatt_co641
soilmu_a_co642 muaggatt_co642
soilmu_a_co643 muaggatt_co643
soilmu_a_co644 muaggatt_co644
soilmu_a_co645 muaggatt_co645
soilmu_a_co651 muaggatt_co651
soilmu_a_co653 muaggatt_co653


In [5]:
result = []
joined_list=[]
for spatial in polygons:
    for table in tables:
        if spatial [-5:] == table [-5:]:
            p = gpd.read_file('D:\PythonProgramming\lab1.gpkg', layer= spatial)
            t = gpd.read_file('D:\PythonProgramming\lab1.gpkg', layer= table)
            t = t.drop(columns = 'geometry')
            result= pd.merge (left = p, right =t, left_on='MUSYM', right_on='musym')
            result['mapunit'] = (table[-5:]) 
            joined_list.append(result)
final = pd.concat(joined_list, axis = 0)

In [6]:
print(final)

     SPATIALVER MUSYM   MUKEY  Shape_Length  Shape_Area  \
0           4.0   AaB   94035      0.010144    0.000004   
1           4.0   AaB   94035      0.011057    0.000004   
2           4.0   AaB   94035      0.010620    0.000005   
3           4.0   AaB   94035      0.015856    0.000013   
4           4.0   AaB   94035      0.025435    0.000025   
..          ...   ...     ...           ...         ...   
405         3.0    49  512721      0.013377    0.000009   
406         3.0    15  512687      0.057632    0.000130   
407         3.0    11  512683      0.072986    0.000325   
408         3.0    26  512698      0.030541    0.000027   
409         3.0    64  512736      0.012214    0.000007   

                                              geometry musym aws025wta  \
0    MULTIPOLYGON (((-104.42220 39.93400, -104.4248...   AaB      4.15   
1    MULTIPOLYGON (((-104.27200 39.78590, -104.2749...   AaB      4.15   
2    MULTIPOLYGON (((-104.34210 39.89100, -104.3428...   AaB      4.1

In [7]:
wbd= gpd.read_file('D:\PythonProgramming\lab1.gpkg', layer = 'wbdhu8_lab1')

In [8]:
intersect = gpd.sjoin(wbd,final, op='intersects', how='left')

In [9]:
print(intersect.groupby(by = 'NAME').count())

              STATES  Shape_Length_left  Shape_Area_left  HUC8  geometry  \
NAME                                                                       
Big Thompson    1979               1979             1979  1979      1979   
St. Vrain       2073               2073             2073  2073      2073   

              index_right  SPATIALVER  MUSYM  MUKEY  Shape_Length_right  \
NAME                                                                      
Big Thompson         1979        1979   1979   1979                1979   
St. Vrain            2073        2073   2073   2073                2073   

              Shape_Area_right  musym  aws025wta  aws0150wta  drclassdcd  \
NAME                                                                       
Big Thompson              1979   1979       1979        1979        1898   
St. Vrain                 2073   2073       2073        2073        1980   

              mapunit  
NAME                   
Big Thompson     1979  
St. Vrain        